In [1]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install deepface

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tf-keras

Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
from deepface import DeepFace

# Configuration constants
CASCADE_PATH = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
SCALE_FACTOR = 1.1
MIN_NEIGHBORS = 5
MIN_FACE_SIZE = (30, 30)

def initialize_video_capture():
    """Initialize the video capture device."""
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise RuntimeError("Error: Could not open video capture.")
    return cap

def detect_faces(face_cascade, frame):
    """Detect faces in the given frame."""
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray_frame, 
        scaleFactor=SCALE_FACTOR, 
        minNeighbors=MIN_NEIGHBORS, 
        minSize=MIN_FACE_SIZE
    )
    return faces

def analyze_emotions(face_roi):
    """Analyze emotions using DeepFace."""
    try:
        result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)
        return result[0]['dominant_emotion']
    except Exception as e:
        print(f"Emotion analysis error: {e}")
        return "Unknown"

def main():
    # Load face cascade classifier
    face_cascade = cv2.CascadeClassifier(CASCADE_PATH)

    # Start capturing video
    cap = initialize_video_capture()

    try:
        while True:
            # Capture frame-by-frame
            ret, frame = cap.read()
            if not ret:
                print("Error: Unable to read frame.")
                break

            # Detect faces
            faces = detect_faces(face_cascade, frame)

            for (x, y, w, h) in faces:
                # Extract and validate the face ROI
                face_roi = frame[y:y + h, x:x + w]
                if face_roi.size == 0:
                    continue

                # Analyze emotions
                emotion = analyze_emotions(face_roi)

                # Draw rectangle and label around the face
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

            # Display the resulting frame
            cv2.imshow('Real-time Emotion Detection', frame)

            # Press 'q' to exit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        # Release the capture and close all windows
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


/Users/neelkolhe/Downloads/theoffice/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
2025-01-08 09:51:57.644 Python[9267:440407] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
